# HuggingFace Transformers
While spaCy provides good-enough NER cabilitities, the accelerated pace of NLP models in recent years means that we can use pre-trained models that leverage modern machine approaches. [HuggingFace](https://huggingface.co/), a company specializing in open-source models, provides an easy-to-use Python library for applying these models on text contained in Sinopia's RDF and to available full-text.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext lab_black
import datetime
import json
import pickle
import string

import kglab
import pandas as pd
import rdflib
import requests
import helpers
import widgets

from transformers import pipeline

## Loading Sinopia Stage RDF Text DataFrame

In [ ]:
stage_text_nodes = pd.read_json("data/stage-text-nodes.json")

In [ ]:
stage_text_nodes.sample(10)

## Huggingface NER Pipeline
The HuggingFace transformers library provides very easy-to-use pipelines for running common NLP tasks like NER. We will create a NER pipeline and run the following *summary* value.

In [ ]:
print(stage_text_nodes.iloc[3566].summary)

In [ ]:
ner_pipe = pipeline("ner")

In [ ]:
ner_results = ner_pipe(stage_text_nodes.iloc[3566].summary)

In [ ]:
for row in ner_results:
    print(row)

The results returned from the [HuggingFace][HUG] NER contain more information and are structured differently than [spaCy][SPACY]. In the [HuggingFace][HUG] pipeline, there are only four classes of entities:

- `I-PER` for a Person name
- `I-ORG` for an Organization name
- `I-LOC` for a location
- `I-MISC` for a Miscellaneous entity. 

The HuggingFace NER also gives a statistical score on how the model is confident that it matched an entity. Also, the [HuggingFace][HUG] NER pipeline results does some character masking (seen as `##` in the results) for many of the entities.

[HUG]: https://huggingface.co/
[SPACY]: https://spacy.io/

## Huggingface Summarization Pipeline
Another [HuggingFace][HUG] pipeline is the *summarization* task that takes a large document and automatically summarizes the text. The pipeline leverages a [Bart](https://arxiv.org/abs/1910.13461) model that was fine-tuned on a CNN / Daily Mail data set.

[HUG]: https://huggingface.co/

In [ ]:
summarizer = pipeline("summarization")

In the previous notebook we looked a *Alarmingly suspicious* that is cataloged in Sinopia at https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c. This resource does not have an abstract or BIBFRAME Summary, so let us read in sections of the full-text to this [HuggingFace][HUG] summizer and see if can autogenerate a summary and add it to this RDF graph.

[HUG]: https://huggingface.co/

In [ ]:
with open("data/65a2b059-5ac1-48a6-adbb-870712c3060c.txt") as fo:
    example1_text = fo.read()

Let's see what happens if we send the entire full-text to the `summarizer`:

In [ ]:
summary_result = summarizer(example1_text)

The maximum number of tokens that the `summarizer` pipeline can process at a time is **1024** while our full-text has **20,348** tokens. Let us try breaking down our large text into smaller "chunks", send each chunk into the `summarizer` pipeline, capture the resulting summary, and at the end, see if the summaries make sense.

First, we will create a list of all of the words in the full-text and then send and summarize 

In [ ]:
example1_words = example1_text.split()
print(f"Total words in full-text {len(example1_words):,}")

In [ ]:
summaries = []
start = 0
size = 500
end = size

start_time = datetime.datetime.utcnow()
print(f"Started at {start_time}")
for i in range(22):  # 10,570 / 500 ~= 21
    if end > len(example1_words):
        end = None
    print(f"{i+1} {start:,} to {end:,} words")
    text_chunk = ' '.join(example1_words[start:end]).encode("ascii", errors="ignore").decode().replace(
    "#", ""
)
    result = summarizer(' '.join(example1_words[start:end]), max_length=75)
    summaries.append(result[0].get('summary_text'))
    start += size
    end += size
end_time = datetime.datetime.utcnow()
print(f"Finished at {end}, total time {(end-start).seconds / 60.} minutes")

In [ ]:
with open("data/summaries.pkl", "wb+") as fo:
    pickle.dump(summaries, fo)

With each summary being limited to 75 words, we still have a large summary that we may want to reduce even further. 

In [ ]:
summary_all = " ".join(summaries)

In [ ]:
all_summary_words = summary_all.split()

In [ ]:
summarizer(" ".join(all_summary_words[0:500]))

In [ ]:
summarizer(" ".join(all_summary_words[500:]))

## Illustration of Cataloging Workflow
To illustrate a possible use of summarization, we will add a BIBFRAME Summary to the original RDF graph for this work.

In [ ]:
example1_result = requests.get(
    "https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c"
)
example1_graph = rdflib.Graph()
for ns, url in helpers.NAMESPACES.items():
    example1_graph.namespace_manager.bind(ns, url)
example1_graph.parse(
    data=json.dumps(example1_result.json().get("data")), format="json-ld"
)
work_uri = rdflib.URIRef(
    "https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c"
)
print(f"Total triples {len(example1_graph)}")

In [ ]:
summary_literal = rdflib.Literal(
    """The Lady of Lyons, The Studio, The Vow of the Omani, The Brigands of Calabria, The Serf The Poacher's Doom, The Hunter of the Alp- Thirty-Three Next Birthday . The work is in the public domain, meaning users are free to copy, use, and redistribute the work in part or in whole. The play is founded on incidents which actually occured during the war of the Rebellion . It introduces Ohio’s brave and gallant McPherson . It abounds with the most beautiful tableaux, drill, marches, scenes upon the battle, in AndersonviHe ."""
)
summary_bnode = rdflib.BNode()
example1_graph.add((work_uri, helpers.BIBFRAME.summary, summary_bnode))
example1_graph.add((summary_bnode, rdflib.RDF.type, helpers.BIBFRAME.Summary))
example1_graph.add((summary_bnode, rdflib.RDFS.label, summary_literal))

In [ ]:
print(example1_graph.serialize(format="turtle").decode())

## Exercise 1

From the `stage_text_nodes`, select a series *title*, *label*, or *summary* values and compare the [spaCy][SPACY] 'en_core_web_sm' NER model with the [Huggingface][HUG] NER model results.

[HUG]: https://huggingface.co/
[SPACY]: https://spacy.io/